In [30]:
from ollama import Client
from gtts import gTTS
from playsound import playsound
import os
from pydub import AudioSegment
from pydub.playback import play

# Configure ffmpeg paths - IMPORTANT: Replace with your actual conda path
ffprobe_path = "/home/masih/anaconda3/bin/ffprobe"
AudioSegment.ffprobe = ffprobe_path
os.environ["PATH"] += os.pathsep + os.path.dirname(ffprobe_path)

# Initialize Ollama client
client = Client(host='http://localhost:11434')

system_prompt = (
    "You are a helpful and polite assistant. "
    "Provide clear and concise responses. "
    "Avoid repetition and maintain a friendly tone."
)

conversation_history = []
max_history_length = 5

def generate_response(user_input):
    global conversation_history
    conversation_history.append(f"User: {user_input}")
    
    messages = [{'role': 'system', 'content': system_prompt}]
    for interaction in conversation_history[-max_history_length:]:
        try:
            role_part, content = interaction.split(': ', 1)
            messages.append({
                'role': role_part.lower(),
                'content': content
            })
        except ValueError:
            continue
    
    try:
        response = client.chat(
            model='llama3.2',  # Corrected model name
            messages=messages,
            options={
                'temperature': 0.7,
                'top_p': 0.9,
                'num_predict': 100
            }
        )
        assistant_response = response['message']['content']
    except Exception as e:
        assistant_response = f"Error: {str(e)}"
    
    conversation_history.append(f"Assistant: {assistant_response}")
    return assistant_response

def text_to_speech(text, output_file="response.mp3", lang='en'):
    try:
        if os.path.exists(output_file):
            os.remove(output_file)
        tts = gTTS(text=text, lang=lang)
        tts.save(output_file)
        print(f"Speech saved to {output_file}")
    except Exception as e:
        print(f"Text-to-speech error: {e}")

def play_sound_with_speed(file_path, speed=1.2):
    if not os.path.exists(file_path):
        print("Audio file not found")
        return
    
    try:
        # Try speed-adjusted playback
        audio = AudioSegment.from_file(file_path)
        altered_audio = audio._spawn(
            audio.raw_data,
            overrides={"frame_rate": int(audio.frame_rate * speed)}
        )
        play(altered_audio)
        print("Playing at 1.2x speed")
    except Exception as e:
        print(f"Speed adjustment failed: {str(e)}. Using simple playback.")
        try:
            playsound(file_path)
        except Exception as fallback_e:
            print(f"All playback methods failed: {str(fallback_e)}")

print("Chat with AI (type 'stop' or 'quit' to exit)")
print("Note: First response may take longer while model loads")

while True:
    try:
        user_input = input("\nYou: ")
        if user_input.lower() in ['stop', 'quit']:
            print("Conversation ended")
            break
            
        response = generate_response(user_input)
        print(f"\nAssistant: {response}")
        
        text_to_speech(response)
        play_sound_with_speed("response.mp3")
        
    except KeyboardInterrupt:
        print("\nConversation interrupted")
        break
    except Exception as e:
        print(f"Critical error: {str(e)}")
        break

Chat with AI (type 'stop' or 'quit' to exit)
Note: First response may take longer while model loads



You:  hi



Assistant: Hello! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?
Speech saved to response.mp3
Speed adjustment failed: [Errno 2] No such file or directory: 'ffprobe'. Using simple playback.


/home/masih/anaconda3/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)



You:  I don't know



Assistant: We can start with something simple then. Would you like to play a game, have a fun conversation, or just explore some topics together? I'm here to listen and help if you need it. What's been on your mind lately?
Speech saved to response.mp3
Speed adjustment failed: [Errno 2] No such file or directory: 'ffprobe'. Using simple playback.


/home/masih/anaconda3/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)



You:  quit


Conversation ended
